In [13]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

from src.feature_extraction import wav_to_features


In [14]:
import os
from pathlib import Path

# 1) Find project root: go up from notebooks/ to repo root
NOTEBOOK_DIR = Path.cwd()                      # when you open Jupyter from project root
if NOTEBOOK_DIR.name == "notebooks":
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    # if you're not inside notebooks/, still handle it safely:
    # assume repo root is current dir or walk up until "notebooks" exists
    p = NOTEBOOK_DIR
    while p != p.parent and not (p / "notebooks").exists():
        p = p.parent
    PROJECT_ROOT = p

# 2) Auto-detect dataset location (supports a couple common layouts)
CANDIDATES = [
    PROJECT_ROOT / "dataset" / "archive",
    PROJECT_ROOT / "dataset",
    PROJECT_ROOT / "data" / "ravdess",
    PROJECT_ROOT / "ravdess",
]

RAVDESS_ROOT = next((c for c in CANDIDATES if c.exists()), None)
if RAVDESS_ROOT is None:
    raise FileNotFoundError(
        f"Could not find RAVDESS dataset folder. Checked: {CANDIDATES}\n"
        f"Project root detected as: {PROJECT_ROOT}"
    )

# 3) Output directory
OUTPUT_DIR = PROJECT_ROOT / "notebooks" / "data"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 4) Collect all wav files (recursive)
wav_files = [str(p) for p in RAVDESS_ROOT.rglob("*.wav")]

print("Project root:", PROJECT_ROOT)
print("RAVDESS_ROOT:", RAVDESS_ROOT)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("Total WAV files:", len(wav_files))
print("Example file:", wav_files[0] if wav_files else "None")



Project root: /Users/malakmaher/Documents/GitHub/Emotion-Recognition
RAVDESS_ROOT: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive
OUTPUT_DIR: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/notebooks/data
Total WAV files: 2781
Example file: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive/Actor_16/03-01-05-01-02-01-16.wav


In [15]:
EMOTION_LABELS = [
    "neutral", "calm", "happy", "sad",
    "angry", "fearful", "disgust", "surprised"
]

emotion_map = {
    1: 0,  # neutral
    2: 1,  # calm
    3: 2,  # happy
    4: 3,  # sad
    5: 4,  # angry
    6: 5,  # fearful
    7: 6,  # disgust
    8: 7   # surprised
}


In [16]:
import os

wav_files = []
for root, _, files in os.walk(str(RAVDESS_ROOT)):
    for f in files:
        if f.endswith(".wav"):
            wav_files.append(os.path.join(root, f))

print("Total WAV files:", len(wav_files))
print("Example:", wav_files[0] if wav_files else "None")


Total WAV files: 2781
Example: /Users/malakmaher/Documents/GitHub/Emotion-Recognition/dataset/archive/Actor_16/03-01-05-01-02-01-16.wav


In [17]:
# wav_files and emotion_map already defined
X, y = [], []

for wav_path in tqdm(wav_files):
    filename = os.path.basename(wav_path)
    emotion_code = int(filename.split("-")[2])
    label = emotion_map[emotion_code]

    feat = wav_to_features(wav_path)  # (128,128,2)
    X.append(feat)
    y.append(label)

X = np.array(X, dtype=np.float32)  # (N,128,128,2)
y = np.array(y, dtype=np.int64)

print("X shape:", X.shape)
print("y shape:", y.shape)

# Dataset-level normalization (per channel)
mean = X.mean(axis=(0, 1, 2), keepdims=True)  # (1,1,1,2)
std  = X.std(axis=(0, 1, 2), keepdims=True) + 1e-6

X = (X - mean) / std

# Save mean/std for prediction consistency
stats = {
    "mean": mean.reshape(-1).tolist(),
    "std": std.reshape(-1).tolist()
}

os.makedirs("notebooks/data", exist_ok=True)
np.save("notebooks/data/X.npy", X)
np.save("notebooks/data/y.npy", y)

import json
with open("notebooks/data/mean_std.json", "w") as f:
    json.dump(stats, f)

print("Saved: X.npy, y.npy, mean_std.json")


100%|██████████| 2781/2781 [00:36<00:00, 76.64it/s]


X shape: (2781, 128, 128, 2)
y shape: (2781,)
Saved: X.npy, y.npy, mean_std.json
